# Task - Part 1

### Step 3 - Import your analysis libraries, as well as your new customer data set as a dataframe

In [1]:
#importing libraries

import pandas as pd
import numpy as np
import os
import datetime as dt #this is to specifically handle with date_joined object and look at it as a date!

In [2]:
#setting up path variables
data_path = r'../2_Data/'

In [3]:
#importing most recent pickle file into working dataframe
df_customers = pd.read_csv(os.path.join(data_path, '1_Original_Data', 'customers.csv'))

### Step 4 - Wrangle the data so that it follows consistent logic; for example, rename columns with illogical names and drop columns that don’t add anything to your analysis

In [4]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [5]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


Looking at the fields, I've decided to:
* Drop the columns with the client names. It's a sensitive ethical issue, especially because there is extra personal info regarding dependants, family status, income, etc. Statistics will only be produced in aggregation. The user_id, however, shall remain because it will be the key to combine with orders dataset.
* Rename fields that use Capital letters to all minor letters, to make it uniform during programming. In the final stage of reporting, I can always re-label for something prettier
* rename `n_dependants` and `fam_status` to more meaningful names

In [6]:
#dropping unneeded fields
df_customers.drop(labels=['First Name','Surnam'], axis=1, inplace=True)

In [7]:
#renaming fields
df_customers.rename(columns = {'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age', 'n_dependants' : 'number_of_dependants', 'fam_status' : 'family_status'}, inplace = True)

In [8]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   user_id               206209 non-null  int64 
 1   gender                206209 non-null  object
 2   state                 206209 non-null  object
 3   age                   206209 non-null  int64 
 4   date_joined           206209 non-null  object
 5   number_of_dependants  206209 non-null  int64 
 6   family_status         206209 non-null  object
 7   income                206209 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


### Step 5 - Complete the fundamental data quality and consistency checks you’ve learned throughout this Achievement; for example, check for and address missing values and duplicates, and convert any mixed-type data.

##### 5.1 - Main verifications

In [9]:
#summary statistics for numeric fields
df_customers.describe()

,user_id,age,number_of_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


1. **user_id** all statistics match the expected, the count is the same value as we had from orders df as unique user_ids, the mean is exactly in the middle of the observations and equal to the median (Q50)
1. **age** it makes sense the min to be 18, most likely is the legal age for a person to be instcart customer. Nothing seems wrong in the remaining statistics
2. **number_of_dependants** all the statistics here make sense as well. At most 3 dependants, an average matching a value well in the middle of the range (0,3). I'll run some value_counts ahead to check more things
3. **income** the max value looks odd, especially when we see that it's super far away from quartiles evolution, and that mean and median are close to each other, not showing signs of skewness. deserves more investigation. In particular I'll check for values beyond 1.5 times the IQR

In [10]:
#checking for columns with mixed datatypes
col_with_mixed_types = False
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].map(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)
    col_with_mixed_types = True
if (not col_with_mixed_types):
  print('No columns with mixed datatypes found')

No columns with mixed datatypes found


In [11]:
#checking if there are null values to be addressed
df_customers.isnull().sum()

user_id                 0
gender                  0
state                   0
age                     0
date_joined             0
number_of_dependants    0
family_status           0
income                  0
dtype: int64

<div class="alert alert-block alert-info">
I already knew this from the summary provided by <code>df_customers.info()</code></div>



In [12]:
#checking for duplicates
df_dup_customers = df_customers[df_customers.duplicated()]
df_dup_customers

,user_id,gender,state,age,date_joined,number_of_dependants,family_status,income


<div class="alert alert-block alert-success">
No duplicates found
</div>

##### 5.2 - Additional consistency checks

In [13]:
df_customers['family_status'].value_counts(dropna=False)

family_status
married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: count, dtype: int64

In [14]:
df_customers['state'].value_counts(dropna=False)

state
Florida                 4044
Colorado                4044
Illinois                4044
Alabama                 4044
District of Columbia    4044
Hawaii                  4044
Arizona                 4044
Connecticut             4044
California              4044
Indiana                 4044
Arkansas                4044
Alaska                  4044
Delaware                4044
Iowa                    4044
Idaho                   4044
Georgia                 4044
Wyoming                 4043
Mississippi             4043
Oklahoma                4043
Utah                    4043
New Hampshire           4043
Kentucky                4043
Maryland                4043
Rhode Island            4043
Massachusetts           4043
Michigan                4043
New Jersey              4043
Kansas                  4043
South Dakota            4043
Minnesota               4043
Tennessee               4043
New York                4043
Washington              4043
Louisiana               4043
Montana 

**Comment** : This looks like it was artificially generated data. It's heavily unlikely that InstaCart database had the same (or almost the same) number of customers for all states!

In [15]:
df_customers['gender'].value_counts(dropna=False)

gender
Male      104067
Female    102142
Name: count, dtype: int64

In [16]:
df_customers['date_joined'].min()

'1/1/2017'

In [17]:
df_customers['date_joined'].max()

'9/9/2019'

From other investigations, I know this max value is wrong, because at this stage, date_joined is an object, and it's being sorted like a string, providing a wrong value (there are dates in the year 2020)

##### Memory saving handling

In [18]:
#memory saving mitigation measures - reducing datatypes to the minimum possible according to the content of each field
#For this dataframe is not much relevant, because it occupies only some MB, but later when merging with the orders df will be important

df_customers['user_id']=df_customers['user_id'].astype('int32')
df_customers['age']=df_customers['age'].astype('int8')
df_customers['number_of_dependants']=df_customers['number_of_dependants'].astype('int8')
df_customers['income']=df_customers['income'].astype('int32')

#additionally, to be able to handle dates properly, I'm transforming date_joined field. 
#This doesn't save memory, but doesn't waste it as well
df_customers['date_joined']=df_customers['date_joined'].astype('datetime64[ns]')

In [19]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   user_id               206209 non-null  int32         
 1   gender                206209 non-null  object        
 2   state                 206209 non-null  object        
 3   age                   206209 non-null  int8          
 4   date_joined           206209 non-null  datetime64[ns]
 5   number_of_dependants  206209 non-null  int8          
 6   family_status         206209 non-null  object        
 7   income                206209 non-null  int32         
dtypes: datetime64[ns](1), int32(2), int8(2), object(3)
memory usage: 8.3+ MB


In [20]:
df_customers['date_joined'].min()

Timestamp('2017-01-01 00:00:00')

In [21]:
df_customers['date_joined'].max()

Timestamp('2020-04-01 00:00:00')

With the field `date_joined` effectively as `datetime64` type, and with the library `datetime` the results are now correct 

##### Additional investigation about outliers in income

By applying the rule Upper Outliers as $$Q75 + 1.5 * (Q75 - Q25) = 124244 + 1.5 * (124244 - 59874) = 220799$$

In [22]:
df_customers.loc[df_customers['income'] >= 220799]

,user_id,gender,state,age,date_joined,number_of_dependants,family_status,income
34,117740,Female,Oregon,55,2017-01-01,1,married,292759
434,159362,Female,Missouri,74,2017-01-03,3,married,372334
818,15683,Male,New Mexico,40,2017-01-05,1,married,251211
979,200930,Male,South Carolina,60,2017-01-06,1,married,300913
991,136298,Male,New Mexico,47,2017-01-06,3,married,433206
...,...,...,...,...,...,...,...,...
204513,21667,Male,Tennessee,53,2020-03-22,0,single,518856
204598,5839,Male,New Jersey,52,2020-03-23,3,married,275506
205458,62924,Male,South Carolina,75,2020-03-28,1,married,245958
206105,5519,Female,Georgia,78,2020-04-01,3,married,262610


875 customers can be found beyond this limit. While they can be considered as top-top earners (they are approximately the 0.3% of highest incomes), I'll proceed with them in the study, because global consumption patterns might reveal something.

### Step 6 - Combine your customer data with the rest of your prepared Instacart data.

Before proceeding here, it should be noted that I returned to this script to further clean up the dataframes I want to merge.

When I arrived to the final task of the project, I've thoroughly listed the business questions I wanted to answer, checked which fields had the data for those answers, and came back to clean up the dataframes as much as I could, dropping all unnecessary fields.

In a realistic scenario, I would probably not be facing the memory restrictions I'm facing in this challenge (would have a better computer), and would leave the fields in place to analyse further, if the stakeholders would wish so.

In [23]:
#clean up of the customers dataframe
df_customers.drop(['gender','date_joined','family_status'],axis=1,inplace=True)

In [24]:
#load dataframe of orders_products
#to be noted here that the pickle file with a 70% random slice of the full orders_products dataset is being used here, 
#due to memory restrictions. This was done while walking through the exercise 4.9

#options to load either the 70% slice or the full dataframe, keep commented the line that is not desired
df_orders_products_merged = pd.read_pickle(os.path.join(data_path, '2_Prepared_Data', 'orders_products_sliced_70pc.pkl'))
#df_orders_products_merged = pd.read_pickle(os.path.join(data_path, '2 Prepared Data', 'orders_products_labeled_v3.pkl'))

In [25]:
#clean up of the orders_products dataframe

df_orders_products_merged.drop(['busy','busy_days','busiest_period_of_day','product_name'],axis=1,inplace=True)

In [26]:
#further optimization of other field datatypes

df_orders_products_merged['department_id'] = df_orders_products_merged['department_id'].astype('int8')
df_orders_products_merged['prices'] = df_orders_products_merged['prices'].astype('float32')
df_orders_products_merged['median_days_between_orders'] = df_orders_products_merged['median_days_between_orders'].astype(pd.Float32Dtype())

In [27]:
df_orders_products_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22682953 entries, 1 to 32403717
Data columns (total 16 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   order_id                          int32  
 1   user_id                           int32  
 2   customer_sequential_order_number  int8   
 3   orders_day_of_week                int8   
 4   order_hour_of_day                 int8   
 5   days_since_prior_order            Int8   
 6   product_id                        int32  
 7   department_id                     int8   
 8   prices                            float32
 9   price_range_loc                   int8   
 10  max_order                         int8   
 11  loyalty_flag                      int8   
 12  avg_price_per_user                float64
 13  high_spender_flag                 bool   
 14  median_days_between_orders        Float32
 15  frequency_flag                    int8   
dtypes: Float32(1), Int8(1), bool(1), float3

It's confirmed that `user_id` (the field that will be used to combine both dataframes) has the same type.

In [28]:
#merging the 70% slice with the customers
df_ord_prod_cust = df_orders_products_merged.merge(df_customers, on='user_id')

In [29]:
#quick check on how many records remain
df_ord_prod_cust.shape

(22682953, 20)

In [30]:
df_ord_prod_cust['user_id'].nunique()

206195

In [31]:
#exporting the resulting dataframe
df_ord_prod_cust.to_pickle(os.path.join(data_path, '2_Prepared_Data', 'orders_products_customers.pkl'))

### Alternative section

In this section, there's a demonstration of how to attempt a variant of the plan B on memory to deal with the full dataframe.
This attempt tries to do the merge in smaller chuncks, and store them step by step in the output .csv file

The following algorithm was inspired in a solution from ChatGPT, with the prompt:

*i have a big pandas dataframe in memory and want to merge it in smaller pieces with another dataframe, and then store those partial merges into a csv file. show me python code to do this.*

The code below is all commented to avoid being run when running the full script. Take off comments on code lines, if you want to experiment.

I tried in a machine of 8GB RAM total, around 2GB available, and it worked. Took more than 40 minutes, and generated a csv file with a total size of 22.7GB. Not useful, I'll proceed with the merge using the pickle file with 70% slice of the ordered items.

In [32]:
# Define the chunk size
# Adjustable based on available memory

"""chunk_size = 10000"""

'chunk_size = 10000'

In [33]:
# Create an iterator over orders_products dataframe

"""num_rows = len(df_orders_products_merged)"""

'num_rows = len(df_orders_products_merged)'

In [34]:
#create an empty dataframe just for the definition of the columns and write it to a new csv file

"""
df_result = pd.DataFrame(columns=(df_orders_products_merged.columns.append(df_customers.columns)).unique())
df_result.to_csv(os.path.join(data_path, '2 Prepared Data', "orders_products_customers.csv"),index_label=False)
"""

'\ndf_result = pd.DataFrame(columns=(df_orders_products_merged.columns.append(df_customers.columns)).unique())\ndf_result.to_csv(os.path.join(data_path, \'2 Prepared Data\', "orders_products_customers.csv"),index_label=False)\n'

In [35]:
"""

for i in range(0, num_rows, chunk_size):
    # Get the chunk
    df_chunk = df_orders_products_merged.iloc[i:i+chunk_size]

    # Perform the merge (adjust how="inner"/"outer"/"left"/"right" as needed)
    df_merged_chunk = pd.merge(df_chunk, df_customers, on="user_id", how="inner")

    # Write to CSV, appending after the first chunk
    df_merged_chunk.to_csv(os.path.join(data_path, '2 Prepared Data', 'orders_products_customers.csv'), mode='a', header=False, index=False)

"""

'\n\nfor i in range(0, num_rows, chunk_size):\n    # Get the chunk\n    df_chunk = df_orders_products_merged.iloc[i:i+chunk_size]\n\n    # Perform the merge (adjust how="inner"/"outer"/"left"/"right" as needed)\n    df_merged_chunk = pd.merge(df_chunk, df_customers, on="user_id", how="inner")\n\n    # Write to CSV, appending after the first chunk\n    df_merged_chunk.to_csv(os.path.join(data_path, \'2 Prepared Data\', \'orders_products_customers.csv\'), mode=\'a\', header=False, index=False)\n\n'